# <font color=blue><div align="center">Seats Allocation within an Aircraft</div></font>



## Modules

In [15]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# Module relatif à Gurobi
from gurobipy import *

# Module csv
import csv

## Préprocessing des données

Pour la suite de notre étude, on considère le modèle d'avion **Airbus A320**, qui dispose de **174** sièges. (La classe businees n'est pas prise en compte)


<img src="modele_avion.jpg" alt="Modèle de l'avion" width="400" height="600">

In [16]:
number_of_seats = 174

In [17]:
# Load the Excel file into a pandas DataFrame

df_21Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=0 , skipfooter=2)
df_22Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=1 , skipfooter=2)
df_23Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=2 , skipfooter=2)
df_24Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=3 , skipfooter=2)
df_30Oct = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=4 , skipfooter=2)
df_05Nov = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=5 , skipfooter=2)
df_07Nov = pd.read_excel('../ST7 - AirFrance/DataSeating 2024.xlsx', sheet_name=6 , skipfooter=2)


In [18]:
# On choisit la df sur laquelle on va travailler

df = df_21Oct

In [19]:
df.head()

,Numéro du groupe,Femmes,Hommes,WCHR,TransitTime
0,1,NaN,1.0,NaN,01:15:00
1,2,1.0,NaN,NaN,00:00:00
2,3,NaN,1.0,NaN,00:00:00
3,4,1.0,NaN,NaN,00:00:00
4,5,NaN,2.0,NaN,05:25:00


In [20]:
def convert_TransitTime(time_val):

    # Convert time to minutes
    total_minutes = time_val.hour * 60 + time_val.minute
    
    # Check if total minutes is greater than 2 hours or equal to 0
    if total_minutes > 120 or total_minutes == 0:
        return float('inf')  # Return infinity
    else:
        return total_minutes

# Apply the function to the 'Time' column and create a new column 'Minutes'
df['TransitTime'] = df['TransitTime'].apply(convert_TransitTime)


In [21]:
df

,Numéro du groupe,Femmes,Hommes,WCHR,TransitTime
0,1,NaN,1.0,NaN,75.0
1,2,1.0,NaN,NaN,inf
2,3,NaN,1.0,NaN,inf
3,4,1.0,NaN,NaN,inf
4,5,NaN,2.0,NaN,inf
...,...,...,...,...,...
106,107,1.0,NaN,NaN,inf
107,108,2.0,NaN,NaN,inf
108,109,2.0,NaN,NaN,inf
109,110,2.0,NaN,NaN,95.0


In [22]:
# Converting time to minutes 

def convert_transit_time(time_str):
    
    # Split the time string and convert to hours, minutes, and seconds
    hours, minutes, seconds = map(int, time_str.split(':'))
    
    # Convert time to minutes
    total_minutes = hours * 60 + minutes
    
    # Check if total minutes is greater than 2 hours
    if total_minutes > 120:
        return float('inf')  # Return infinity
    else:
        return total_minutes


In [23]:
# Groups : dict[int : List[int]]

Passagers = dict()
i = 1

for group in df.itertuples():
    if not pd.isna(group[2]):
        for k in range(int(group[2])):
            Passagers[i] = {'gender': 0, 'group':group[1], 'transit':group[5]}
            i+=1
    if not pd.isna(group[3]):
        for k in range(int(group[3])):
            Passagers[i] = {'gender': 1, 'group':group[1], 'transit':group[5]}
            i+=1
    if not pd.isna(group[4]):
        for k in range(int(group[4])):
            Passagers[i] = {'gender': 2, 'group':group[1], 'transit':group[5]}
            i+=1
    


    
Passagers  

{1: {'gender': 1, 'group': 1, 'transit': 75.0},
 2: {'gender': 0, 'group': 2, 'transit': inf},
 3: {'gender': 1, 'group': 3, 'transit': inf},
 4: {'gender': 0, 'group': 4, 'transit': inf},
 5: {'gender': 1, 'group': 5, 'transit': inf},
 6: {'gender': 1, 'group': 5, 'transit': inf},
 7: {'gender': 0, 'group': 6, 'transit': inf},
 8: {'gender': 0, 'group': 7, 'transit': inf},
 9: {'gender': 0, 'group': 8, 'transit': 80.0},
 10: {'gender': 1, 'group': 8, 'transit': 80.0},
 11: {'gender': 1, 'group': 9, 'transit': inf},
 12: {'gender': 0, 'group': 10, 'transit': inf},
 13: {'gender': 0, 'group': 11, 'transit': inf},
 14: {'gender': 1, 'group': 11, 'transit': inf},
 15: {'gender': 1, 'group': 12, 'transit': 70.0},
 16: {'gender': 1, 'group': 13, 'transit': inf},
 17: {'gender': 0, 'group': 14, 'transit': inf},
 18: {'gender': 1, 'group': 14, 'transit': inf},
 19: {'gender': 1, 'group': 15, 'transit': inf},
 20: {'gender': 1, 'group': 16, 'transit': 95.0},
 21: {'gender': 0, 'group': 17, 'tr

## Définition du modèle

In [24]:
number_of_seats = 174
weight_f = 70
weight_m = 85
weight_h = 92.5
number_of_rows = 29
number_of_columns = 6

**Définition des Variables $X_{(i,j,k)}$**

In [25]:
# m: Model

m = Model("Seats_Allocation")

AssignmenVarDict = {(i, j, k) : m.addVar(vtype = GRB.BINARY, name=f'passager_{i}_{j}_{k}') for i in range(1, number_of_rows + 1) for j in range(1, number_of_columns + 1) for k in range(1, len(Passagers) + 1)}


**Contraintes satisfaction clients**

In [26]:
# Contrainte : touts les passagers ont un siège 

m.addConstr(quicksum([AssignmenVarDict[(i, j, k)] for i in range(1, number_of_rows + 1) for j in range(1, number_of_columns+1) for k in range(1, len(Passagers) + 1)]) == len(Passagers), name = 'Passagers Sièges')

#Contrainte : Siège par personne

SeatPerPassengerCONST = {k : m.addConstr(quicksum([AssignmenVarDict[(i, j, k)] for i in range(1, number_of_rows + 1) for j in range(1, number_of_columns)]) == 1, name = f'Unique Siège Passager {k}') for k in range(1, len(Passagers) + 1)}

**Contrainte Centrage de l'avion**

In [ ]:
PondeMen = weight_m*quicksum([AssignmenVarDict[(i,j,k)]*(i,j) for i in range(1, number_of_rows + 1) for j in range(1, number_of_columns+1) for k in range(1, len(Passagers) + 1) if Passagers[k]['gender'] == 1]) 
PondeWomen = weight_f*quicksum([AssignmenVarDict[(i,j,k)]*(i,j) for i in range(1, number_of_rows + 1) for j in range(1, number_of_columns+1) for k in range(1, len(Passagers) + 1) if Passagers[k]['gender'] == 0]) 
PondeWCHR= weight_h*quicksum([AssignmenVarDict[(i,j,k)]*(i,j) for i in range(1, number_of_rows + 1) for j in range(1, number_of_columns+1) for k in range(1, len(Passagers) + 1) if Passagers[k]['gender'] == 2]) 

Barycentre = (PondeMen + PondeWCHR + PondeWomen)/